# Prueba Cardali Python

In [35]:
import pandas as pd
import os 
from pathlib import Path 

In [92]:
root_dir = Path(".").resolve().parent
filename = "PRUEBA ANALISTA PYTHON.xlsx"
file_path = os.path.join(root_dir, "Data", "Raw", filename)
file_path

'C:\\Users\\calde\\OneDrive\\Escritorio\\Cardali\\Data\\Raw\\PRUEBA ANALISTA PYTHON.xlsx'

In [93]:
Data_inicial = pd.read_excel(file_path,sheet_name="FORMATO INICIAL")
Data_inicial.head(2)

,Agent #,Agent Name,Policy ID,Individual Name,Policy: Policy Number,Policy: Coverage Type,Policy: Status,Policy: Effective Date,BFD,Alloc Date,Revenue Type,Premium,Num Lives,Rate,Commission Rec'd,Net Commission,Carrier: Carrier Name,Source
0,88888860,ADRIANA AGUIRRE,NaN,SALCEDO OSPINO LEONOR,G80866824650008FL086,NaN,NaN,NaN,2023-10-01,NaN,Override,NaN,3,6,18,18,Humana,Import
1,88888860,ADRIANA AGUIRRE,NaN,MARTELL COLLAZO LEIDIARY,G80877050370008TX086,NaN,NaN,NaN,2023-10-01,NaN,Override,NaN,1,6,6,6,Humana,Import


In [94]:
Data_final = pd.read_excel(file_path,sheet_name="FORMATO FINAL")
Data_final.head(2)

,Individual: First Name,Individual: Last Name,Carrier: Carrier Name,Policy: Coverage Type,Policy: Policy Number,Policy: Status,Policy: Effective Date,Agent: External ID,Agent: First Name,Agent: Last Name,Agency: Agency Name
0,LEONOR,SALCEDO OSPINO,Humana,Health (ACA),G80866824650008FL086,Active,NaN,88888860,ADRIANA,AGUIRRE,CARDALI
1,LEIDIARY,MARTELL COLLAZO,Humana,Health (ACA),G80877050370008TX086,Active,NaN,88888860,ADRIANA,AGUIRRE,CARDALI


In [95]:
Data_agentes = pd.read_excel(file_path,sheet_name="BASE AGENTES")
Data_agentes.head(2)

,Agent: External ID,Agent:Name,Agency: Agency Name
0,88866040,JOSE PEREZ,ANT INSURANCE
1,88636886,LEYDI TORRES,ANT INSURANCE


## Comparar dimensiones

In [96]:
print("Las dimensiones del dataframe inicial son ", Data_inicial.shape,"\n", 
      "Las dimensiones del dataframe final son ", Data_final.shape)

Las dimensiones del dataframe inicial son  (919, 18) 
 Las dimensiones del dataframe final son  (919, 11)


## Comparar columnas

In [80]:
# Obtener listas de nombres de columnas
columnas_inicial = list(Data_inicial.columns)
columnas_final = list(Data_final.columns)

In [81]:
# Encontrar columnas en Data_final que provienen de Data_inicial
columnas_no_provenientes = set(columnas_final) - set(columnas_inicial)

# Encontrar columnas en Data_final que provienen de Data_inicial
columnas_provenientes = set(columnas_final).intersection(set(columnas_inicial))

### Transformo a dataframe

In [97]:
df_columnas_provenientes = pd.DataFrame(columnas_provenientes,columns=["provenientes"])
df_columnas_provenientes

,provenientes
0,Policy: Status
1,Carrier: Carrier Name
2,Policy: Coverage Type
3,Policy: Policy Number
4,Policy: Effective Date


In [98]:
df_columnas_no_provenientes = pd.DataFrame(columnas_no_provenientes,columns=["No provenientes"])
df_columnas_no_provenientes

,No provenientes
0,Agent: External ID
1,Individual: Last Name
2,Agent: Last Name
3,Individual: First Name
4,Agent: First Name
5,Agency: Agency Name


## Creación columnas
- Individual:First Name
- Individual:Last Name

In [99]:
#Crear copy dataframe original
Data_final_copy = Data_inicial.copy()

In [100]:
# Dividir la columna 'individual name' en palabras
Data_final_copy['Individual Name'] = Data_final_copy['Individual Name'].str.split()
Data_final_copy['Individual Name']

0                 [SALCEDO, OSPINO, LEONOR]
1              [MARTELL, COLLAZO, LEIDIARY]
2      [PINA, HERNANDEZ, MARIANNY, Beatriz]
3      [PINA, HERNANDEZ, MARIANNY, Beatriz]
4      [PINA, HERNANDEZ, MARIANNY, Beatriz]
                       ...                 
914               [CORTES, GUZMAN, JOSE, E]
915                 [COSTLY, HARRY, Daniel]
916                 [COSTLY, HARRY, Daniel]
917       [BAZAN, CABELLOS, SEGUNDO, ISAAC]
918       [BAZAN, CABELLOS, SEGUNDO, ISAAC]
Name: Individual Name, Length: 919, dtype: object

In [101]:
# Crear nuevas columnas 'individual:first name'
Data_final_copy['individual:first name'] = Data_final_copy['Individual Name'].apply(lambda x: x[-1] if len(x) == 3 or len(x) == 2 else x[-2] if len(x) == 4 or len == 5 else None)

In [102]:
# Convertir la lista en una cadena
Data_final_copy['Individual Name'] = Data_final_copy['Individual Name'].apply(lambda x: ' '.join(x))

In [103]:
# Aplicar str.replace para quitar la parte de la Individual Name que coincide con la individual:first name
Data_final_copy['Individual Name'] = Data_final_copy.apply(lambda row: row['Individual Name'].replace(row['individual:first name'], '') if pd.notna(row['Individual Name']) and pd.notna(row['individual:first name']) else row['Individual Name'], axis=1)

In [104]:
# Resultado final
Data_final_copy[['individual:first name', 'Individual Name']]

,individual:first name,Individual Name
0,LEONOR,SALCEDO OSPINO
1,LEIDIARY,MARTELL COLLAZO
2,MARIANNY,PINA HERNANDEZ Beatriz
3,MARIANNY,PINA HERNANDEZ Beatriz
4,MARIANNY,PINA HERNANDEZ Beatriz
...,...,...
914,JOSE,CORTES GUZMAN E
915,Daniel,COSTLY HARRY
916,Daniel,COSTLY HARRY
917,SEGUNDO,BAZAN CABELLOS ISAAC


- Agent: First Name
- Agent: Last Name

In [105]:
# Dividir la columna 'Agent Name' en palabras
Data_final_copy['Agent: First Name'] = Data_final_copy['Agent Name'].str.split().str[0]
Data_final_copy['Agent: Last Name'] = Data_final_copy['Agent Name'].str.split().str[-1]

In [106]:
Data_final_copy[['Agent: First Name', 'Agent: Last Name']]

,Agent: First Name,Agent: Last Name
0,ADRIANA,AGUIRRE
1,ADRIANA,AGUIRRE
2,ADRIANA,AGUIRRE
3,ADRIANA,AGUIRRE
4,ADRIANA,AGUIRRE
...,...,...
914,YURI,CASTANEDA
915,ZAIDY,PADRINO
916,ZAIDY,PADRINO
917,ZAIDY,PADRINO


- Agent: External ID
- Agency: Agency Name

In [107]:
# Combinar DataFrames usando 'Agent:Name' como clave
result = pd.merge(Data_final_copy, Data_agentes[['Agent:Name', 'Agent: External ID', 'Agency: Agency Name']],
                  left_on='Agent #', right_on='Agent: External ID', how='left')

In [110]:
result.columns

Index(['Agent #', 'Agent Name', 'Policy ID', 'Individual Name',
       'Policy: Policy Number', 'Policy: Coverage Type', 'Policy: Status',
       'Policy: Effective Date', 'BFD', 'Alloc Date', 'Revenue Type',
       'Premium', 'Num Lives', 'Rate', 'Commission Rec'd', 'Net Commission',
       'Carrier: Carrier Name', 'Source', 'individual:first name',
       'Agent: First Name', 'Agent: Last Name', 'Agent:Name',
       'Agent: External ID', 'Agency: Agency Name'],
      dtype='object')

In [111]:
# Seleccionar las columnas deseadas
df_subset = result[['individual:first name', 'Individual Name', 'Carrier: Carrier Name', 
                'Policy: Coverage Type', 'Policy: Policy Number', 'Policy: Status', 
                'Policy: Effective Date', 'Agent #','Agent: First Name', 'Agent: Last Name', 
                'Agency: Agency Name']]

In [114]:
# Renombrar las columnas seleccionadas
df_subset = df_subset.rename(columns={'Individual Name': 'individual:last name', 'Agent #': 'Agent: External ID'})
df_subset


,individual:first name,individual:last name,Carrier: Carrier Name,Policy: Coverage Type,Policy: Policy Number,Policy: Status,Policy: Effective Date,Agent: External ID,Agent: First Name,Agent: Last Name,Agency: Agency Name
0,LEONOR,SALCEDO OSPINO,Humana,NaN,G80866824650008FL086,NaN,NaN,88888860,ADRIANA,AGUIRRE,CARDALI
1,LEIDIARY,MARTELL COLLAZO,Humana,NaN,G80877050370008TX086,NaN,NaN,88888860,ADRIANA,AGUIRRE,CARDALI
2,MARIANNY,PINA HERNANDEZ Beatriz,Humana,NaN,G80866822880008TX086,NaN,NaN,88888860,ADRIANA,AGUIRRE,CARDALI
3,MARIANNY,PINA HERNANDEZ Beatriz,Humana,NaN,G80866822880008TX086,NaN,NaN,88888860,ADRIANA,AGUIRRE,CARDALI
4,MARIANNY,PINA HERNANDEZ Beatriz,Humana,NaN,G80866822880008TX086,NaN,NaN,88888860,ADRIANA,AGUIRRE,CARDALI
...,...,...,...,...,...,...,...,...,...,...,...
914,JOSE,CORTES GUZMAN E,Humana,NaN,G80877522860008TX086,NaN,NaN,88630324,YURI,CASTANEDA,SERVIPLUS
915,Daniel,COSTLY HARRY,Humana,NaN,G80877485830008NC086,NaN,NaN,20888883,ZAIDY,PADRINO,SERVIPLUS
916,Daniel,COSTLY HARRY,Humana,NaN,G80877485830008NC086,NaN,NaN,20888883,ZAIDY,PADRINO,SERVIPLUS
917,SEGUNDO,BAZAN CABELLOS ISAAC,Humana,NaN,G80877864480008NC086,NaN,NaN,20888883,ZAIDY,PADRINO,SERVIPLUS


In [109]:
out_name = "reporte.csv"  
out_path = os.path.join(root_dir,"Data","Processed", out_name)
result.to_csv(out_path)